In [10]:
#Habilitar intellisense
%config IPCompleter.greedy=True

# Representación (feature engineering)
Una de las representaciones más usuales consiste en convertir un documento en un vector con valores numéricos con tantas variables/columnas como tokens existan en el vocabulario de problema.

Los valores en el arreglo pueden ser por ejemplo: la frequencia del tóken en el documento, el código en un word embedding, o la métrica TF-IDF.

Hay escenarios en los cuales estos vectores pueden tener un gran tamaño, en estos casos se recomienda emplear alguna estrategia para reducir la dimensionalidad de los vectores (Ej. feature hashing, locality sensitive hashing).

## Obtener el vocabulario del problema

In [9]:
# https://www.lostiempos.com/deportes/multideportivo/20200115/olympic-albert-einstein-ucb-lpz-van-paso-firme-liga-superior
texto = """Los clubes cochabambinos de Olympic, Albert Einstein y el paceño Universidad Católica Boliviana (UCB) avanzan a paso firme y constante rumbo a la corona en la Liga Superior de voleibol, rama femenina, que se desarrolla en el coliseo Julio Borelli Vitterito de La Paz, luego de cosechar sendas victorias la noche de este martes. El ganador será representante de Bolivia en la Liga Sudamericana de Clubes 2020.

El campeón defensor del título, Olympic, superó 3-0  a su verdugo de la final de la edición 2017, el también cochabambino San Simón. La victoria para las olympiquistas fue con sets de 25-14, 25-13 y 25-14."""
import nltk
import string
import numpy as np
import pandas as pd
import collections
import math
spanishStopWords = set( nltk.corpus.stopwords.words('spanish') + list(string.punctuation))
spanishStopWords

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste

In [3]:
#https://www.nltk.org/api/nltk.tokenize.html
#Return a sentence-tokenized copy of text, using NLTK’s recommended sentence tokenizer 
#(currently PunktSentenceTokenizer for the specified language).
documents = nltk.tokenize.sent_tokenize(text=texto, language='spanish')
len(documents)
documents[0]

'Los clubes cochabambinos de Olympic, Albert Einstein y el paceño Universidad Católica Boliviana (UCB) avanzan a paso firme y constante rumbo a la corona en la Liga Superior de voleibol, rama femenina, que se desarrolla en el coliseo Julio Borelli Vitterito de La Paz, luego de cosechar sendas victorias la noche de este martes.'

In [4]:
def normalize_document(document, language='spanish'):
    tokens = nltk.word_tokenize(document,language)
    filtered_tokens = [token.lower() for token in tokens if token not in spanishStopWords]
    normalized_document = " ".join(filtered_tokens);
    return normalized_document
 
normalize_document(documents[0])

'los clubes cochabambinos olympic albert einstein paceño universidad católica boliviana ucb avanzan paso firme constante rumbo corona liga superior voleibol rama femenina desarrolla coliseo julio borelli vitterito la paz luego cosechar sendas victorias noche martes'

In [5]:
def normalize_corpus(documents, language='spanish'):
    return np.array([normalize_document(document, language) for document in documents])

normalized_corpus = normalize_corpus(documents)
#type(normalized_corpus)
normalized_corpus

array(['los clubes cochabambinos olympic albert einstein paceño universidad católica boliviana ucb avanzan paso firme constante rumbo corona liga superior voleibol rama femenina desarrolla coliseo julio borelli vitterito la paz luego cosechar sendas victorias noche martes',
       'el ganador representante bolivia liga sudamericana clubes 2020',
       'el campeón defensor título olympic superó 3-0 verdugo final edición 2017 cochabambino san simón',
       'la victoria olympiquistas sets 25-14 25-13 25-14'], dtype='<U264')

In [6]:
def get_problem_vocabulary(normalized_corpus):
    all_tokens = [] 
    for document in normalized_corpus:
        all_tokens.extend(document.split())  
    #[all_tokens.extend(document.split()) for document in normalized_corpus]
    all_tokens_sorted = sorted(set(all_tokens))
    
    token_and_position = {}
    for i, token in enumerate(all_tokens_sorted):
        token_and_position[token] = i
    
    return token_and_position
    
problem_vocabulary = get_problem_vocabulary(normalized_corpus)
#len(problem_vocabulary)
problem_vocabulary

{'2017': 0,
 '2020': 1,
 '25-13': 2,
 '25-14': 3,
 '3-0': 4,
 'albert': 5,
 'avanzan': 6,
 'bolivia': 7,
 'boliviana': 8,
 'borelli': 9,
 'campeón': 10,
 'católica': 11,
 'clubes': 12,
 'cochabambino': 13,
 'cochabambinos': 14,
 'coliseo': 15,
 'constante': 16,
 'corona': 17,
 'cosechar': 18,
 'defensor': 19,
 'desarrolla': 20,
 'edición': 21,
 'einstein': 22,
 'el': 23,
 'femenina': 24,
 'final': 25,
 'firme': 26,
 'ganador': 27,
 'julio': 28,
 'la': 29,
 'liga': 30,
 'los': 31,
 'luego': 32,
 'martes': 33,
 'noche': 34,
 'olympic': 35,
 'olympiquistas': 36,
 'paceño': 37,
 'paso': 38,
 'paz': 39,
 'rama': 40,
 'representante': 41,
 'rumbo': 42,
 'san': 43,
 'sendas': 44,
 'sets': 45,
 'simón': 46,
 'sudamericana': 47,
 'superior': 48,
 'superó': 49,
 'título': 50,
 'ucb': 51,
 'universidad': 52,
 'verdugo': 53,
 'victoria': 54,
 'victorias': 55,
 'vitterito': 56,
 'voleibol': 57}

## one-hot encoding

In [9]:
normalized_corpus[2]

'el campeón defensor título olympic superó 3-0 verdugo final edición 2017 cochabambino san simón'

In [8]:
def one_hot_vector(document, problem_vocabulary):
    vector = np.zeros(len(problem_vocabulary),dtype=int)
    for token in document.split():
        vector[problem_vocabulary[token]] = 1
    
    return vector

one_hot_vector(normalized_corpus[2], problem_vocabulary)

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0])

In [14]:
normalized_corpus[3]

'la victoria olympiquistas sets 25-14 25-13 25-14'

## TF  (Term Frequency)

In [15]:
normalized_corpus

array(['los clubes cochabambinos olympic albert einstein paceño universidad católica boliviana ucb avanzan paso firme constante rumbo corona liga superior voleibol rama femenina desarrolla coliseo julio borelli vitterito la paz luego cosechar sendas victorias noche martes',
       'el ganador representante bolivia liga sudamericana clubes 2020',
       'el campeón defensor título olympic superó 3-0 verdugo final edición 2017 cochabambino san simón',
       'la victoria olympiquistas sets 25-14 25-13 25-14'], dtype='<U264')

In [16]:
#https://docs.python.org/2/library/collections.html#collections.Counter
count_by_token = collections.Counter(normalized_corpus[3].split())
count_by_token

Counter({'la': 1,
         'victoria': 1,
         'olympiquistas': 1,
         'sets': 1,
         '25-14': 2,
         '25-13': 1})

In [73]:
problem_vocabulary

{'2017': 0,
 '2020': 1,
 '25-13': 2,
 '25-14': 3,
 '3-0': 4,
 'albert': 5,
 'avanzan': 6,
 'bolivia': 7,
 'boliviana': 8,
 'borelli': 9,
 'campeón': 10,
 'católica': 11,
 'clubes': 12,
 'cochabambino': 13,
 'cochabambinos': 14,
 'coliseo': 15,
 'constante': 16,
 'corona': 17,
 'cosechar': 18,
 'defensor': 19,
 'desarrolla': 20,
 'edición': 21,
 'einstein': 22,
 'el': 23,
 'femenina': 24,
 'final': 25,
 'firme': 26,
 'ganador': 27,
 'julio': 28,
 'la': 29,
 'liga': 30,
 'los': 31,
 'luego': 32,
 'martes': 33,
 'noche': 34,
 'olympic': 35,
 'olympiquistas': 36,
 'paceño': 37,
 'paso': 38,
 'paz': 39,
 'rama': 40,
 'representante': 41,
 'rumbo': 42,
 'san': 43,
 'sendas': 44,
 'sets': 45,
 'simón': 46,
 'sudamericana': 47,
 'superior': 48,
 'superó': 49,
 'título': 50,
 'ucb': 51,
 'universidad': 52,
 'verdugo': 53,
 'victoria': 54,
 'victorias': 55,
 'vitterito': 56,
 'voleibol': 57}

In [ ]:
from heapq import nlargest

In [18]:
freq = collections.Counter(['a','b','c','a'])
freq

In [20]:
nlargest(10, freq, key=freq.get)

['a', 'b', 'c']

In [17]:
def term_frequency_vector(document, problem_vocabulary):
    vector = np.zeros(len(problem_vocabulary),dtype=int)
    
    count_by_token = collections.Counter(document.split());
    for token, count in count_by_token.items():
        vector[problem_vocabulary[token]] = count
    
    return vector

term_frequency_vector(normalized_corpus[3], problem_vocabulary)

array([0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [21]:
freq = nltk.probability.FreqDist(['a','b','c','a'])
freq

FreqDist({'a': 2, 'b': 1, 'c': 1})

In [22]:
nlargest(10, freq, key=freq.get)

['a', 'b', 'c']

###  Evite escribir código, cuando se puede hacer con librerias


__[Scikit-Learn Design Principles](https://towardsdatascience.com/scikit-learn-design-principles-d1371958059b)__

In [93]:
normalized_corpus_simple = np.array(['p1 co','p2 co co'])
normalized_corpus_simple

array(['p1 co', 'p2 co co'], dtype='<U8')

In [94]:
# CountVectorizer implementa Transformer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer() #Parametros con valores por defecto
cv_matrix = cv.fit_transform(normalized_corpus_simple) #combina fit y luego trasform
cv_matrix

<2x3 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [95]:
print("sparse matrix:")
print(cv_matrix)
print("regular np array:")
print(cv_matrix.toarray())
print("vocabulary:")
print(cv.vocabulary_)
print("variables:")
print(cv.get_feature_names())
print("one-hot encoding:")
print(np.sign(cv_matrix.toarray()))

sparse matrix:
  (0, 0)	1
  (0, 1)	1
  (1, 2)	1
  (1, 0)	2
regular np array:
[[1 1 0]
 [2 0 1]]
vocabulary:
{'p1': 1, 'co': 0, 'p2': 2}
variables:
['co', 'p1', 'p2']
one-hot encoding:
[[1 1 0]
 [1 0 1]]


In [6]:
pd.DataFrame(cv_matrix.toarray(), columns=cv.get_feature_names())

,co,p1,p2
0,1,1,0
1,2,0,1


In [96]:
normalized_corpus_simple

array(['p1 co', 'p2 co co'], dtype='<U8')

In [97]:
#bi-gramas
bv = CountVectorizer(ngram_range=(1,2))
cv_matrix_with_bigrams = bv.fit_transform(normalized_corpus_simple)
print("sparse matrix:")
print(cv_matrix_with_bigrams)
print("regular np array:")
print(cv_matrix_with_bigrams.toarray())
print("variables:")
print(bv.get_feature_names())

sparse matrix:
  (0, 3)	1
  (0, 0)	1
  (0, 2)	1
  (1, 1)	1
  (1, 5)	1
  (1, 4)	1
  (1, 0)	2
regular np array:
[[1 0 1 1 0 0]
 [2 1 0 0 1 1]]
variables:
['co', 'co co', 'p1', 'p1 co', 'p2', 'p2 co']


In [125]:
pd.DataFrame(cv_matrix_with_bigrams.toarray(), columns=bv.get_feature_names())

,co,co co,p1,p1 co,p2,p2 co
0,1,0,1,1,0,0
1,2,1,0,0,1,1


## IDF  (Inverse Document Frequency)
idf(t) = log(cantidad-documentos/cantidad-documentos-con-el-termino)

Min = 1; A valores más alejados de 1, mayor IDF

Interpretación de resultados

In [46]:
normalized_corpus_simple = np.array(['p1 co','p2 co co'])
normalized_corpus_simple

array(['p1 co', 'p2 co co'], dtype='<U8')

In [33]:
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
from sklearn.feature_extraction.text import TfidfVectorizer
tfIdfv = TfidfVectorizer(norm='l2')
tfIdfv.fit(normalized_corpus_simple)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [35]:
# Qué pasa con el valor de idf para las palabras que aparecen en muchos documentos?
# Debido a cómo sckit-learn hace el cálculos, no hay IDF = 0, el valor mínimo es 1
dict(zip(tfIdfv.get_feature_names(),tfIdfv.idf_ ))

{'co': 1.0, 'p1': 1.4054651081081644, 'p2': 1.4054651081081644}

In [120]:
normalized_corpus_simple = np.array(['p1 co','p2 co co'])
tfIdfv.fit(normalized_corpus_simple)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [121]:
#No deberían los valores ser mayores a 1? ver norm=None en la documentación
#Este score/indicador aumenta con la frequencia de la palabra en el documento
#Pero disminuye cuando la palabra se hace muy común (baja su relevancia)
#Si los stop words no se quitaran, los valores de este indicador serían elevados/reducidos?
tt_matrix = tfIdfv.transform(normalized_corpus_simple)
print("regular np array:")
print(tt_matrix.toarray())
print("variables:")
print(tfIdfv.get_feature_names())

regular np array:
[[0.57973867 0.81480247 0.        ]
 [0.81818021 0.         0.57496187]]
variables:
['co', 'p1', 'p2']
